In [2]:
from typing import Iterator

def lcg(seed: int, m: int=2**32, a: int=1664525, c: int=1013904223) -> Iterator[int]:
    """
    This generator implements the Linear Congruential Generator algorithm
    :param m: the modulus, a positive integer constant
    :param a: the multiplier, a non-negative integer constant < m
    :param c: the increment, a non-negative integer constant < m
    :param seed: the starting state of the LCG. It is used to initialize the pseudo-random number sequence
    :return: a non-negative integer in [0, m-1] representing the i-th state of the generator
    """
    x = seed
    while True:
        x = (a * x + c) % m
        yield x/m

def xorshift_64(seed: int) -> Iterator[int]:
    """
    This generator implements the XORShift algorithm.
    :param seed: the initial state of the generator. Should be a non-zero integer.
    :return: pseudo-random integers generated by the XORShift algorithm.
    """
    x = seed if seed != 0 else 1
    while True:
        x ^= (x << 13) & 0xFFFFFFFFFFFFFFFF
        x ^= (x >> 7) & 0xFFFFFFFFFFFFFFFF
        x ^= (x << 17) & 0xFFFFFFFFFFFFFFFF
        x &= 0xFFFFFFFFFFFFFFFF
        yield x/0xFFFFFFFFFFFFFFFF


def rotl(x: int, k: int) -> int:
    return ((x << k) | (x >> (64 - k))) & 0xFFFFFFFFFFFFFFFF

def xoshiro_64(seed: list[int]) -> Iterator[int]:
    """
    xoshiro256++ generator
    :param seed: list of 4 uint64 integers, the internal state (cant be all 0)
    :return: pseudo-random 64-bit integers
    """
    s = seed.copy()
    if len(s) != 4 or all(x == 0 for x in s):
        raise ValueError("Seed must be a list of 4 non-zero uint64 integers")

    while True:
        result = (rotl(s[0] + s[3], 23) + s[0]) & 0xFFFFFFFFFFFFFFFF
        t = (s[1] << 17) & 0xFFFFFFFFFFFFFFFF

        s[2] ^= s[0]
        s[3] ^= s[1]
        s[1] ^= s[2]
        s[0] ^= s[3]

        s[2] ^= t
        s[3] = rotl(s[3], 45)

        yield result/0xFFFFFFFFFFFFFFFF


In [1]:

from math import log, sin, pi

def Poisson_adelgazamiento_mejorado(T, generator):
    interv = [4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48]
    lamda = [12.5, 27.5, 35, 35, 27.5, 12.5, 12.5, 27.5, 35, 35, 27.5, 12.5]
    j = 0
    t =-log (1 - next(generator) ) / lamda[j]
    Eventos = []
    while t <= T:
        if t <= interv[j]:
            V = next(generator)
            if V < 20 + 15*sin((t-6)*2*pi/24) / lamda[j]:
                Eventos.append(t)
                t +=-log(1- next(generator)) / lamda[j]
        else:
            t = interv[j] + (t- interv[j]) * lamda[j] / lamda[j + 1]
            j += 1
    return Eventos

In [6]:
eventos = Poisson_adelgazamiento_mejorado(48, lcg(123456789))
eventos[0:20]

[0.019293436236225956,
 0.07873760046224713,
 0.14172197832399192,
 0.16899607290770907,
 0.20919381110430366,
 0.25393836619528487,
 0.28583959118274527,
 0.38043490136823793,
 0.6858184329892387,
 0.7479685458266172,
 0.8114066762432373,
 0.8580452559511279,
 0.9550442283031874,
 0.9960288955105823,
 1.02320926596493,
 1.0451369685211938,
 1.115369101771122,
 1.1703368446339706,
 1.174612272033769,
 1.1753340647409616]

In [11]:
from random import random
def atender(mu:int=35)->float:
    return -log(1 - random()) / mu

In [12]:
def simular_atencion(eventos:list)->dict:
    current_time = 0

    tiempos_inicio = []
    tiempos_espera = []
    tiempos_salida = []

    for llegada in eventos:
        comienzo = max(llegada, current_time)
        servicio = atender()
        salida = comienzo + servicio

        tiempos_inicio.append(comienzo)
        tiempos_espera.append(comienzo - llegada)
        tiempos_salida.append(salida)

        current_time = salida

    return {
        "tiempos_llegada": eventos,
        "tiempos_inicio": tiempos_inicio,
        "tiempos_salida": tiempos_salida,
        "tiempos_espera": tiempos_espera,
    }

In [13]:
arrivals = [
 0.019293436236225956,
 0.07873760046224713,
 0.14172197832399192,
 0.16899607290770907,
 0.20919381110430366,
 0.25393836619528487,
 0.28583959118274527,
 0.38043490136823793,
 0.6858184329892387,
 0.7479685458266172,
 0.8114066762432373,
 0.8580452559511279,
 0.9550442283031874,
 0.9960288955105823
]

resultado = simular_atencion(arrivals)

for i in range(len(arrivals)):
    print(f"Cliente {i+1}:")
    print(f"  Llegada: {resultado['tiempos_llegada'][i]:.4f}")
    print(f"  Inicio atención: {resultado['tiempos_inicio'][i]:.4f}")
    print(f"  Salida: {resultado['tiempos_salida'][i]:.4f}")
    print(f"  Espera en cola: {resultado['tiempos_espera'][i]*60:.2f} min")
    print()

Cliente 1:
  Llegada: 0.0193
  Inicio atención: 0.0193
  Salida: 0.0215
  Espera en cola: 0.00 min

Cliente 2:
  Llegada: 0.0787
  Inicio atención: 0.0787
  Salida: 0.0867
  Espera en cola: 0.00 min

Cliente 3:
  Llegada: 0.1417
  Inicio atención: 0.1417
  Salida: 0.1509
  Espera en cola: 0.00 min

Cliente 4:
  Llegada: 0.1690
  Inicio atención: 0.1690
  Salida: 0.2273
  Espera en cola: 0.00 min

Cliente 5:
  Llegada: 0.2092
  Inicio atención: 0.2273
  Salida: 0.2329
  Espera en cola: 1.09 min

Cliente 6:
  Llegada: 0.2539
  Inicio atención: 0.2539
  Salida: 0.2547
  Espera en cola: 0.00 min

Cliente 7:
  Llegada: 0.2858
  Inicio atención: 0.2858
  Salida: 0.3329
  Espera en cola: 0.00 min

Cliente 8:
  Llegada: 0.3804
  Inicio atención: 0.3804
  Salida: 0.4256
  Espera en cola: 0.00 min

Cliente 9:
  Llegada: 0.6858
  Inicio atención: 0.6858
  Salida: 0.6873
  Espera en cola: 0.00 min

Cliente 10:
  Llegada: 0.7480
  Inicio atención: 0.7480
  Salida: 0.7584
  Espera en cola: 0.00 min
